# Regresores. Aproximación inicial y explicabilidad del modelo

En este notebook se realizará una aproximación inicial con un conjunto de datos reducido y se utilizará la IA explicable para comprender el modelo.

In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDRegressor
from explainerdashboard import RegressionExplainer, ClassifierExplainer, ExplainerDashboard
import plotly.express as px
from math import sqrt

RANDOM_STATE = 123

Cargamos los datos del mes de septiembre para SMT y Celonis.

In [2]:
celonis_df = pd.read_excel('data/20230929132544_CELONIS_EXPORT.xlsx')

In [3]:
smt_df = pd.read_excel('data/08_I11_SMT_Incidents_of_own_solution_group(s).xlsx')

smt_df.columns = smt_df.iloc[11].values
smt_df = smt_df.iloc[12:]
smt_df = smt_df.reset_index(drop=True)

c:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default


In [4]:
smt_df

,Incident ID,Original Incident Number,Requisition ID,Created Date (UTC+0),Incident Type,Status,Status Reason,Company,Customer Department,Region,...,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Resolution Product Name,Primary Center Code,Target Date,Notes,Resolution,Created by CI-Hotline
0,INC000024939028,NaN,NaN,27.06.2023 12:04,User Service Restoration,Closed,No Further Action Required,GS,GS/HRS2-ES,EMEA,...,- None -,End Device,Personal Computer,Standard Notebook Touch,NaN,933357,02.08.2023 08:00,Q: PROBLEM DESCRIPTION? A:barcode appears wron...,"Se comunica con el usuario, nos indica cerrar ...",Yes
1,INC000025114665,NaN,NaN,17.07.2023 11:27,User Service Request,Closed,No Further Action Required,PS,RBEF/LOG,EMEA,...,- None -,Service,Infrastructure Service,Support Service,NaN,983311,,Su solicitud vía e-mail ha sido transferida a ...,La usuaria nos indica que se ha podido resolve...,Yes
2,INC000025172062,NaN,NaN,25.07.2023 06:59,User Service Request,Closed,No Further Action Required,AA,AA/CTG1-EU,EMEA,...,- None -,End Device,Personal Computer,Standard Notebook Touch,NaN,933411,01.08.2023 09:00,Q: User ID (e.g. xyz8fe): A:LVM5MD Q: Compute...,"Se cambia la placa base, se reinstala el equip...",Yes
3,INC000025184174,NaN,NaN,26.07.2023 09:17,User Service Request,Closed,No Further Action Required,GR,FCM2-Md,EMEA,...,- None -,End Device,Personal Computer,Standard Desktop Tiny,NaN,933481,,Please update software local in FCM departame...,El usuario indica que cerremos la incidencia y...,No
4,INC000025197112,NaN,NaN,27.07.2023 14:20,User Service Request,Closed,No Further Action Required,C,C/CGR-IB2,EMEA,...,- None -,Service,Application Service,Application Service,NaN,NaN,15.08.2023 22:00,Q: User ID (e.g. xyz8fe): A: GOJ5MD Q: Compu...,"Todo el proceso a funcionado correctamente, la...",Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,INC000025416771,NaN,NaN,31.08.2023 15:01,User Service Request,Resolved,No Further Action Required,PS,RBEF/HRL,EMEA,...,- None -,Service,Application Service,Application Service,- None -,983221,,Only for local personal support team Request,"Dear customer, Por favor evalúe el servic...",No
250,INC000025416773,NaN,NaN,31.08.2023 15:02,User Service Restoration,Resolved,No Further Action Required,PS,RBEF/HRL,EMEA,...,- None -,Service,Application Service,Application Service,- None -,983221,,Only for local personal support team Incident,"Dear customer, Por favor evalúe el servic...",No
251,INC000025416876,NaN,NaN,31.08.2023 15:05,User Service Request,Resolved,No Further Action Required,PS,PS/MFT-PLA,EMEA,...,- None -,Service,Business Service,Business Service,- None -,NaN,,Only for local personal support team Request,"Dear customer, You receive this mail as autom...",Yes
252,INC000025416879,NaN,NaN,31.08.2023 15:06,User Service Request,Resolved,No Further Action Required,PS,PS/MFT-PLA,EMEA,...,- None -,Service,Application Service,Application Service,- None -,NaN,,Only for local personal support team Request,"Dear customer, You receive this mail as autom...",Yes


In [5]:
celonis_df

,Incident Number,Submit Date,Service Name,Site Group,Site,Incident Type,Ticket Summary,Submitter Group,Last Assigned Organization,Last Assigned Group,...,Last Resolution Date,Template Name,User Company,Resolver Organization,Requisition ID,Reported Source,Last Resolution Time,CI,Status,Priority
0,INC000025219797,2023-08-01 06:37:22,MOZILLA FIREFOX |EMEA,ES,Md,User Service Request,How to clear cache in Mozilla Firefox manually,ITSD - IT Service Desk,Service Solutions,ITSD - ES,...,2023-08-01 06:37:22,NaN,EU1,CI-Hotline / ITSD 1st Level,NaN,Phone,0.280556,NaN,Closed,Low
1,INC000025219902,2023-08-01 06:48:05,SCCM PACKAGE INSTALLATION FAILURE,ES,Md,User Service Restoration,LEN_Hotkey_9.2.0.5_Tx90_W10 - RBHW9390,ITSD - IT Service Desk,Service Solutions,ITSD - SCCM Installation Support,...,2023-08-21 06:42:56,NaN,C,CI-Hotline / ITSD 2nd Level,NaN,Phone,480.225833,SCCM CLIENT HARDWARE DRIVERS,Closed,Low
2,INC000025221003,2023-08-01 07:36:48,BUSINESS-OBJECTS_APPLICATION,ES,Md1,User Service Restoration,Excel - Analysis Error,ITSD - IT Service Desk,Data and Digital,BI BO Platform support,...,2023-08-01 09:18:30,NaN,AE,Rest Support Organizations,NaN,Phone,1.920278,BUSINESS-OBJECTS_APPLICATION,Closed,Low
3,INC000025221039,2023-08-01 07:48:04,ITSD DOCUMENTED,ES,Md1,User Service Request,Autocad License problem,ITSD - IT Service Desk,Service Solutions,ITSD - ES,...,2023-09-01 12:53:00,NaN,BMG,CI-Hotline / ITSD 1st Level,NaN,Chat,749.219167,NaN,Closed,Medium
4,INC000025221092,2023-08-01 07:59:17,MY COMPUTER - LOCAL SUPPORT |WORLD,ES,Md1,User Service Request,Equipo MD-C-00029 sin espacio libre,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,...,2023-08-01 12:34:45,NaN,AE,IT Personal Support,NaN,Email,4.697778,MD-C-00029,Closed,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,INC000025416134,2023-08-31 13:41:35,WCMS BGN,ES,Md1,User Service Request,[C7] The user cannot enter Sugerencias,ITSD - IT Service Desk,RBEI,CENTRAL inside Portal Advisor RBEI - EXT,...,2023-09-06 05:56:07,NaN,AE,Rest Support Organizations,NaN,Phone,136.461667,NaN,Closed,Low
436,INC000025416227,2023-08-31 13:31:13,IDM - USERADMINISTRATION | ITSP ORDERFORM ACTI...,ES,Md1,User Service Request,Manage access rights via IdM,IT Infrastructure Automation,Service Solutions,ITSD - IDM Expert Team,...,2023-08-31 16:22:53,S2T_Autobus_SNOW_Request,AE,CI-Hotline / ITSD 2nd Level,RITM1414984,Self Service,2.861111,NaN,Closed,Low
437,INC000025416602,2023-08-31 14:20:30,IDENTITIES (USERS) NEW / CHANGE / DELETE,ES,Md,User Service Request,User Prolongation Order RITM1412095 required ...,IT Infrastructure Automation,Digital Workplace,Identity Lifecycle Support,...,2023-09-08 10:35:36,S2T_Autobus_Request,BD,Rest Support Organizations,NaN,Systems Management,188.251667,NaN,Resolved,Low
438,INC000025417081,2023-08-31 16:47:12,SCANSYSTEM,ES,Md1,User Service Restoration,No abre programa Visual Factory Scansistem,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,...,2023-09-01 11:40:07,NaN,AE,IT Personal Support,NaN,Email,18.953333,MD-C-000FP,Closed,Low


Preanálisis de los datos.

In [6]:
celonis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Incident Number             440 non-null    object        
 1   Submit Date                 440 non-null    datetime64[ns]
 2   Service Name                440 non-null    object        
 3   Site Group                  440 non-null    object        
 4   Site                        440 non-null    object        
 5   Incident Type               440 non-null    object        
 6   Ticket Summary              440 non-null    object        
 7   Submitter Group             440 non-null    object        
 8   Last Assigned Organization  440 non-null    object        
 9   Last Assigned Group         440 non-null    object        
 10  Entry Channel               440 non-null    object        
 11  Last Resolution Date        432 non-null    datetime64[ns]

In [7]:
smt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 47 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Incident ID                               254 non-null    object
 1   Original Incident Number                  0 non-null      object
 2   Requisition ID                            18 non-null     object
 3   Created Date (UTC+0)                      254 non-null    object
 4   Incident Type                             254 non-null    object
 5   Status                                    254 non-null    object
 6   Status Reason                             254 non-null    object
 7   Company                                   254 non-null    object
 8   Customer Department                       253 non-null    object
 9   Region                                    254 non-null    object
 10  Site Group                                253 non-

Tratamiento de nulos

In [8]:
celonis_df.isnull().sum()

Incident Number                 0
Submit Date                     0
Service Name                    0
Site Group                      0
Site                            0
Incident Type                   0
Ticket Summary                  0
Submitter Group                 0
Last Assigned Organization      0
Last Assigned Group             0
Entry Channel                   0
Last Resolution Date            8
Template Name                 280
User Company                    0
Resolver Organization           0
Requisition ID                387
Reported Source                14
Last Resolution Time            8
CI                            231
Status                          0
Priority                        0
dtype: int64

In [9]:
celonis_df = celonis_df.drop(['Template Name', 'Requisition ID', 'CI'], axis=1)
celonis_df.dropna(inplace=True)
celonis_df = celonis_df.drop(['Submit Date', 'Last Resolution Date', 'Ticket Summary'], axis=1)

In [10]:
celonis_df.shape

(419, 15)

In [11]:
output = smt_df.isnull().sum()
output.to_csv('data/missing_values_output.csv')
output

Incident ID                                   0
Original Incident Number                    254
Requisition ID                              236
Created Date (UTC+0)                          0
Incident Type                                 0
Status                                        0
Status Reason                                 0
Company                                       0
Customer Department                           1
Region                                        0
Site Group                                    1
Site                                          1
Desk Location                                 1
Reported Source                               0
Summary                                       0
Impact                                        0
Open Priority                                 0
Current Priority                              0
Assigned Group                                0
Assigned Group Department                     0
In Progess Time (hhh:mi)\n              

In [12]:
smt_df = smt_df.drop(['Original Incident Number', 'Requisition ID', 'CI+', 'Product Name', 
                      'Resolution Product Name', 'Primary Center Code'], axis=1)
smt_df.dropna(inplace=True)
smt_df = smt_df.drop(['Created Date (UTC+0)', 'Summary', 'Last Modified Date (UTC+0)', 
                      'First Resolved Date (UTC+0)', 'Last Resolved Date (UTC+0)', 'Notes', 'Resolution'], axis=1)

In [13]:
smt_df.shape

(252, 34)

Combinación de dataframes de Celonis y SMT.

In [14]:
combined_df = pd.merge(celonis_df, smt_df, how='inner', left_on='Incident Number', right_on='Incident ID')
combined_df.drop(['Incident ID'], axis=1, inplace=True)

In [15]:
pd.set_option('display.max_columns', None)
combined_df

,Incident Number,Service Name,Site Group_x,Site_x,Incident Type_x,Submitter Group,Last Assigned Organization,Last Assigned Group,Entry Channel,User Company,Resolver Organization,Reported Source_x,Last Resolution Time,Status_x,Priority,Incident Type_y,Status_y,Status Reason,Company,Customer Department,Region,Site Group_y,Site_y,Desk Location,Reported Source_y,Impact,Open Priority,Current Priority,Assigned Group,Assigned Group Department,In Progess Time (hhh:mi)\n,Actual Duration/Open Time (hhh:mi),Down Time of CI-Unavailability (hhh:mi),Operational Categorization Tier 1,Operational Categorization Tier 2,Operational Categorization Tier 3,Service+,Product Categorization Tier 1,Product Categorization Tier 2,Product Categorization Tier 3,Resolution Categorization Tier 1,Resolution Categorization Tier 2,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Target Date,Created by CI-Hotline
0,INC000025221092,MY COMPUTER - LOCAL SUPPORT |WORLD,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,4.697778,Closed,Low,User Service Request,Closed,No Further Action Required,AE,RBEM/MSE1,EMEA,ES,MD1,Md1 150P2,Email,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,4:21,0004:42,Not Specified,Request,- None -,- None -,MY COMPUTER - LOCAL SUPPORT |WORLD,Service,End Device Service,Personal Computer,Application,Performance,- None -,End Device,Personal Computer,Standard Desktop Tiny,,Yes
1,INC000025221966,MY COMPUTER - LOCAL SUPPORT |WORLD,ES,Md,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,EU2,IT Personal Support,Phone,4.116667,Closed,Low,User Service Request,Closed,No Further Action Required,EU2,EU2/SOE-IB,EMEA,ES,MD,Md,Phone,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,2:11,0004:07,Not Specified,Request,- None -,- None -,MY COMPUTER - LOCAL SUPPORT |WORLD,Service,End Device Service,Personal Computer,Application,Performance,- None -,End Device,Personal Computer,Standard Notebook Touch,,Yes
2,INC000025223159,MY COMPUTER - CONSULTATION |WORLD,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,23.067500,Closed,Low,User Service Request,Closed,No Further Action Required,AE,RBEM/LOM-Q,EMEA,ES,MD1,Md1 155P3,Email,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,4:50,0023:04,Not Specified,Request,- None -,- None -,MY COMPUTER - CONSULTATION |WORLD,Service,Business Service,Business Service,Infrastructure,Hardware,- None -,End Device,Personal Computer,Standard Desktop Tiny,,Yes
3,INC000025223804,LOCAL APPLICATIONS CONSULTING |WORLD,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,22.799722,Closed,Low,User Service Request,Closed,No Further Action Required,AE,RBEM/TEF7,EMEA,ES,MD1,Md1,Email,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,19:14,0022:48,Not Specified,Request,- None -,- None -,LOCAL APPLICATIONS CONSULTING |WORLD,Service,IT Service,IT Service,Application,Performance,- None -,End Device,Personal Computer,Standard Desktop Tiny,,Yes
4,INC000025224626,MY MOBILE - DEVICE REPLACEMENT |ES,ES,Md,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1,IT Service Portal,AA,IT Personal Support,Self Service,24.208611,Closed,Low,User Service Request,Closed,No Further Action Required,AA,AA/SWS2-EI,EMEA,ES,MD,Md,Self Service,4-Minor/Localized,Low,Low,Spain1,BD/PFE-IA,0:4,0024:13,Not Specified,Request,- None -,- None -,MY MOBILE - DEVICE REPLACEMENT |ES,Service,End Device Service,Mobile Device Service,Request,Hardware,- None -,Service,End Device Service,Mobile Device Service,,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

Aplicamos el onehot encoder mediante pd.getdummies().

In [16]:
columns_list = combined_df.columns
columns_list = [elem for elem in columns_list if elem not in ['Incident Number', 'Last Resolution Time']] 
combined_df_one_hot = pd.get_dummies(combined_df, columns=columns_list)
combined_df_one_hot

,Incident Number,Last Resolution Time,Service Name_ACTIVE DIRECTORY GROUP MANAGEMENT |EMEA,Service Name_BITLOCKER,Service Name_CAMERFIRMA,Service Name_CI-CAT ADMINISTRATION |SPAIN1,Service Name_CORASON,Service Name_ENTRUST,Service Name_FILE SHARE AND DFS SERVICES EMEA,Service Name_HANDS-ON SERVICE |SPAIN1,Service Name_IDM - USERADMINISTRATION,Service Name_IT WORKPLACE CONSULTING |WORLD,Service Name_ITSP TICKETS |EMEA W/O DACH,Service Name_LOCAL APPLICATIONS CONSULTING |WORLD,Service Name_MANAGED PRINT SERVICES |CENTRAL,Service Name_MEDTRA,Service Name_MICROSOFT TEAMS,Service Name_MOBILE PIM SERVICE,Service Name_MS OUTLOOK,Service Name_MS OUTLOOK MOBILE,Service Name_MY COMPUTER - CONSULTATION |WORLD,Service Name_MY COMPUTER - DEVICE LOST / STOLEN |WORLD,Service Name_MY COMPUTER - DEVICE NETWORK CONNECTIVITY |WORLD,Service Name_MY COMPUTER - DEVICE REPAIR |WORLD,Service Name_MY COMPUTER - DEVICE RETURN |WORLD,Service Name_MY COMPUTER - LOCAL SUPPORT |WORLD,Service Name_MY MOBILE - CHANGE RATE PLAN |ES,Service Name_MY MOBILE - DEVICE REPLACEMENT |ES,Service Name_MY MOBILE - DEVICE REQUEST |ES,Service Name_MY MOBILE - LOCAL SUPPORT |SPAIN1,Service Name_MY MOBILE - REPAIR DEVICE,Service Name_NON-SCCM SOFTWARE INSTALLATION |EMEA,"Service Name_NT PASSWORD RESET |EMEA, LA",Service Name_OBJETIVOS,Service Name_PDA-MDM |SPAIN1,Service Name_PH1-HR-CORE-ESS,Service Name_SCCM PACKAGE INSTALLATION FAILURE,Service Name_SMALL IT EQUIPMENT |WORLD,Service Name_SOFTWARE NO SCCM |SPAIN1,Service Name_TELEPHONY LOCAL SUPPORT |SPAIN1,Site Group_x_ES,Site_x_Md,Site_x_Md1,Incident Type_x_Infrastructure Restoration,Incident Type_x_User Service Request,Incident Type_x_User Service Restoration,Submitter Group_Human Resources - Personnel Administration,Submitter Group_IT Infrastructure Automation,Submitter Group_ITSD - IT Service Desk,Submitter Group_Spain1,Submitter Group_Spain1 - EXT,Last Assigned Organization_Regional Services EMEA,Last Assigned Group_Spain1,Last Assigned Group_Spain1 - EXT,Entry Channel_AutomationBus,Entry Channel_CI-Hotline / ITSD,Entry Channel_IT Personal Support,Entry Channel_IT Service Portal,Entry Channel_IT Support Page,Entry Channel_Other,User Company_AA,User Company_AE,User Company_BD,User Company_BMG,User Company_BT,User Company_C,User Company_EU1,User Company_EU2,User Company_GR,User Company_GS,User Company_HC,User Company_M,User Company_PT,User Company_SO,Resolver Organization_IT Personal Support,Reported Source_x_Chat,Reported Source_x_Direct Input,Reported Source_x_Email,Reported Source_x_Phone,Reported Source_x_Self Service,Reported Source_x_Systems Management,Reported Source_x_Walk In,Status_x_Closed,Priority_Low,Priority_Medium,Incident Type_y_Infrastructure Restoration,Incident Type_y_User Service Request,Incident Type_y_User Service Restoration,Status_y_Closed,Status_y_Resolved,Status Reason_Customer Follow-Up Required,Status Reason_No Further Action Required,Company_AA,Company_AE,Company_BD,Company_BMG,Company_BT,Company_C,Company_EU1,Company_EU2,Company_GR,Company_GS,Company_HC,Company_M,Company_PT,Company_SO,Customer Department_AA-AS/TSS2-EI,Customer Department_AA/SEI-WTE,Customer Department_AA/SMS121-EU,Customer Department_AA/SOA2-EU,Customer Department_AA/SSP1,Customer Department_AA/SUC-IB,Customer Department_AA/SWS1-SP,Customer Department_AA/SWS2-EI,Customer Department_AE/EIR3,Customer Department_BD/PFE4-IA,Customer Department_BMG-2ES/ENG1,Customer Department_BMG-2ES/ENG3,Customer Department_BMG-2ES/ENG4,Customer Department_BMG-2ES/ICO,Customer Department_BMG-2ES/MOE1,Customer Department_BMG-2ES/MOE2,Customer Department_BMG-2ES/PAS,Customer Department_BMG-2ES/PUL11,Customer Department_BMG-2ES/PUL13,Customer Department_BT-VS/SAL3.2-EU,Customer Department_C/CGR-IB2,Customer Department_C/CMR-IB,Customer Department_C/DSO-IB,Customer Department_C/FIO-EMEA1,Customer Department_C/LSR-IB,Customer Department_EU2/SOE-IB,Customer Department_FCM1-Md,Customer Department_GS/HRS-ES,Customer Department_GS/OBC12-EMEA2,Customer D

In [17]:
combined_df_one_hot.rename(columns=lambda x: x.replace('.', ' '), inplace=True)
combined_df_one_hot.shape

(168, 498)

Separación en conjunto de entrenamiento y prueba.

In [18]:
X = combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).values
y = combined_df_one_hot['Last Resolution Time'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

Creación del modelo de Gradient Boosting y entrenamiento

In [19]:
gbr = GradientBoostingRegressor(random_state=RANDOM_STATE)

gbr.fit(X_train, y_train)

GradientBoostingRegressor(random_state=123)

Realizamos la evaluación del modelo.

In [20]:
predictions = gbr.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

predictions

Error cuadrático medio (MSE) en el conjunto de prueba: 1217.6183
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 34.8944
Error medio absoluto (MAE) en el conjunto de prueba: 14.9610


array([ -0.44090036,  21.99474324,  -0.44090036,  21.99474324,
        79.40287041,  -0.44090036,  23.16529684,  -0.44090036,
         9.94939144,  -0.44090036,  -0.44090036,   7.94445582,
        21.99474324,   7.94445582,  -0.44090036,   7.94445582,
        21.99474324,  -0.44090036,  26.39419482,  -0.44090036,
        -0.44090036,  -0.44090036,  -0.44090036,   7.94445582,
         6.05243695,  20.99020875,  21.99474324,  -0.44090036,
        79.56321462,  -0.44090036, 175.57587321,  26.39419482,
        -0.44090036,   7.94445582])

In [21]:
y_test

array([1.38888889e-03, 5.14741667e+01, 3.05555556e-03, 3.57752778e+01,
       4.08333333e-01, 5.55555556e-04, 4.82841667e+01, 1.50000000e-02,
       3.86111111e-02, 3.13888889e-02, 2.69444444e-02, 2.19444444e-02,
       3.18055556e-01, 2.38888889e-02, 3.91666667e-02, 1.80555556e-02,
       3.79777778e+00, 2.77777778e-04, 2.02827778e+01, 2.77777778e-04,
       2.02777778e-02, 3.33333333e-03, 2.77777778e-04, 3.77777778e-02,
       3.75833333e-01, 4.04769444e+01, 1.50000000e+00, 5.55555556e-04,
       5.70880556e+01, 4.72222222e-03, 5.74166667e-01, 4.22991667e+01,
       5.55555556e-04, 1.83333333e-02])

Evaluación del modelo con el conjunto de prueba para estudiar la existencia de sobreaprendizaje.

In [22]:
predictions_train = gbr.predict(X_train)

mse = mean_squared_error(y_train, predictions_train)
mae = mean_absolute_error(y_train, predictions_train)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

predictions

Error cuadrático medio (MSE) en el conjunto de prueba: 45.5798
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 6.7513
Error medio absoluto (MAE) en el conjunto de prueba: 4.4536


array([ -0.44090036,  21.99474324,  -0.44090036,  21.99474324,
        79.40287041,  -0.44090036,  23.16529684,  -0.44090036,
         9.94939144,  -0.44090036,  -0.44090036,   7.94445582,
        21.99474324,   7.94445582,  -0.44090036,   7.94445582,
        21.99474324,  -0.44090036,  26.39419482,  -0.44090036,
        -0.44090036,  -0.44090036,  -0.44090036,   7.94445582,
         6.05243695,  20.99020875,  21.99474324,  -0.44090036,
        79.56321462,  -0.44090036, 175.57587321,  26.39419482,
        -0.44090036,   7.94445582])

Creación, entrenamiento y evaluación del modelo de descenso del gradiento estocástico.

In [23]:
X = combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).values
y = combined_df_one_hot['Last Resolution Time'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

sgd = SGDRegressor(random_state=RANDOM_STATE)

sgd.fit(X_train, y_train)

predictions = sgd.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

predictions

Error cuadrático medio (MSE) en el conjunto de prueba: 827.0020
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 28.7576
Error medio absoluto (MAE) en el conjunto de prueba: 19.6432


array([  2.06864475,  29.89068788,   2.94995409,  77.34746848,
        46.64719163,   3.97203269,  38.43326361,  -3.21983033,
        14.13765608,   9.35074418,  -5.77453395,   3.16027876,
        23.69900703,   2.17423186,  20.28427001,   3.16027876,
        52.42018737, -18.34339218,  77.53792219, -18.78835194,
         4.34480236,   2.9963999 ,  33.86765225,  -7.22043046,
        -0.3527197 ,  37.26223773,  47.37664721,   3.56169403,
       111.24085391,   2.94995409,  81.7701847 ,  96.6897245 ,
        -6.08070164,  11.54583691])

In [24]:
y_test

array([1.38888889e-03, 5.14741667e+01, 3.05555556e-03, 3.57752778e+01,
       4.08333333e-01, 5.55555556e-04, 4.82841667e+01, 1.50000000e-02,
       3.86111111e-02, 3.13888889e-02, 2.69444444e-02, 2.19444444e-02,
       3.18055556e-01, 2.38888889e-02, 3.91666667e-02, 1.80555556e-02,
       3.79777778e+00, 2.77777778e-04, 2.02827778e+01, 2.77777778e-04,
       2.02777778e-02, 3.33333333e-03, 2.77777778e-04, 3.77777778e-02,
       3.75833333e-01, 4.04769444e+01, 1.50000000e+00, 5.55555556e-04,
       5.70880556e+01, 4.72222222e-03, 5.74166667e-01, 4.22991667e+01,
       5.55555556e-04, 1.83333333e-02])

Creación de los dataframes para poder utilizar explainer dashboard.

In [25]:
X_test_df = pd.DataFrame(X_test, columns=combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).columns)
y_test_df = pd.DataFrame(y_test, columns=['Last Resolution Time'])

Creación de explaiener dashboard para el modelo de Gradient Boosting. Es importante recalcar que la ejecución de la siguiente celda siempre me ha dado un error de Max retries exceeded with url tras un tiempo de ejecución. Aún así, el enlace generado es funcional y el dashboard funciona con normalidad.

In [ ]:
explainer = RegressionExplainer(gbr, X_test_df, y_test_df)

ExplainerDashboard(explainer).run()


Creación de explaiener dashboard para el modelo de Stochastic Gradient Descent.

In [ ]:
explainer = RegressionExplainer(sgd, X_test_df, y_test_df)

ExplainerDashboard(explainer).run(port=8050)

In [ ]:
#pd.reset_option('display.max_columns')
